In [ ]:
import simplejson
import tqdm

import numpy as np
import xarray as xr
import geopandas as gpd

import matplotlib.pyplot as plt

In [ ]:
regions = gpd.read_file("../data/regions.geojson")

with open("../data/metadata.json") as f:
    metadata = simplejson.load(f)

def export(name, out):
    with open(f"../data/{name}.json", "w") as f:
        simplejson.dump(out, f, sort_keys=True, ignore_nan=True)

## Percentiles

In [ ]:
perc = (
    xr.open_dataset("models-dist/Tas_percentiles_final.nc")
    .rename_vars({"temperature_percentile": "tas"})
    .drop_vars(["model_name", "region_name"])
    .astype(np.float64)
)
perc

## Projections

In [ ]:
proj = (
    xr.open_dataset("models-proj/projections.nc")
    .astype(np.float64)
)
proj

In [ ]:
def make_perc(da):
    if da is None:
        return None
    perc = []
    for p in metadata["percentiles"]:
        if p in da["percentile"]:
            perc.append(
                da.sel({"percentile": p})
                .values
                .astype(np.float64)
                .round(2) # TODO adjust for variables?
            )
        else:
            perc.append(None)
    return perc


def make_proj(da, rcp):
    if da is None:
        return None
    proj = [
        da.sel({"scenario": "historical"}).dropna(dim="time").squeeze().values.round(3),
        *da.sel({"scenario": rcp}).dropna(dim="time").squeeze().values.round(3)
    ]
    return proj


data = {}
for _, region in tqdm.tqdm(regions.iterrows()):
    nuts_id = region["id"]
    data[nuts_id] = {}

    for var in ["tas", "pr"]:
        data[nuts_id][var] = []

        for model in metadata["models"]:

            try:
                perc_single = perc[var].sel({
                    "region": nuts_id,
                    "model": "{gcm}*{rcm}".format(**model)
                })
            except KeyError:
                perc_single = None

            try:
                proj_single = proj[var].sel({
                    "region": nuts_id,
                    "model": "{gcm} {rcm} {ens}".format(**model)
                })
            except KeyError:
                proj_single = None

            data[nuts_id][var].append({
                "perc": make_perc(perc_single),
                "rcp26": make_proj(proj_single, "rcp26"),
                "rcp45": make_proj(proj_single, "rcp45"),
                "rcp85": make_proj(proj_single, "rcp85"),
            })

In [ ]:
export("details", data)